In [ ]:
import os
from kafka import KafkaProducer
import json
import schedule
import time

In [ ]:
KAFKA_HOST = # URL
TARGET_HOST = # HOSTNAME

In [ ]:
def is_target_available(target):
    response = os.system('ping -c 1 ' + target)
    return response == 0

In [ ]:
discovery_producer = KafkaProducer(
    bootstrap_servers=[KAFKA_HOST], 
    value_serializer=lambda x: json.dumps(x).encode('utf-8'))

In [ ]:
last_reported_state = False

def handle_discovery():
    global last_reported_state
    state = is_target_available(TARGET_HOST)
    if last_reported_state != state:
        print('State changed:', state, last_reported_state)
        discovery_producer.send('host_discovery', { 'type': 'HOST_DISCOVERY', 'state': state, 'host': TARGET_HOST })
        last_reported_state = state

In [ ]:
job = schedule.every(15).seconds.do(handle_discovery)
while True:
    schedule.run_pending()
    time.sleep(1)
schedule.cancel_job(job)